In [2]:
import cv2
from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import vision

import os
from PIL import Image

2024-01-22 11:33:41.029561: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-22 11:33:41.111489: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 11:33:41.334937: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 11:33:41.336552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 11:33:49.652388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
model = '/bask/homes/f/fspo1218/amber/projects/binary_moth_detector/output_models/gbif_model.tflite'
enable_edgetpu = False
num_threads = 1

In [4]:
base_options = core.BaseOptions(file_name=model, use_coral=enable_edgetpu, num_threads=num_threads)
detection_options = processor.DetectionOptions(max_results=20, score_threshold=0.1)
options = vision.ObjectDetectorOptions(base_options=base_options, detection_options=detection_options)
detector = vision.ObjectDetector.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
def get_detections(detection_result):
    detections_list = []

    for counter, detection in enumerate(detection_result.detections, start=1):
        bounding_box = detection.bounding_box
        origin_x, origin_y, width, height = (
            bounding_box.origin_x, bounding_box.origin_y,
            bounding_box.width, bounding_box.height,
        )

        category_info = detection.categories[0]  # Assuming one category per detection
        category_dict = {
            'index': category_info.index,
            'score': category_info.score,
            'display_name': category_info.display_name,
            'category_name': category_info.category_name,
        }

        detection_dict = {
            'counter': counter,
            'bounding_box': {
                'origin_x': origin_x, 'origin_y': origin_y,
                'width': width, 'height': height,
            },
            'categories': [category_dict],
        }

        detections_list.append(detection_dict)
        
        return(detections_list)

In [10]:
# list images in path
images = os.listdir('/bask/homes/f/fspo1218/amber/data/test_images/raw/')
output_directory = '/bask/homes/f/fspo1218/amber/data/test_images/crop_test/'

for image_path in images:
    image = cv2.imread('/bask/homes/f/fspo1218/amber/data/test_images/raw/' + image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a TensorImage object from the RGB image.
    input_tensor = vision.TensorImage.create_from_array(image)
    
    # Get the inferences 
    detection_result = detector.detect(input_tensor)    
    detections_list = get_detections(detection_result)
    print(detection_result)
    
    counter = 1
    
    # Draw bounding boxes on the image
    for detection in detections_list:
        bounding_box = detection['bounding_box']
        origin_x, origin_y, width, height = (
                bounding_box['origin_x'],
                bounding_box['origin_y'],
                bounding_box['width'],
                bounding_box['height'],
            )

        # Crop the original image
        cropped_image = image[origin_y:origin_y + height, origin_x:origin_x + width]

        # Save the cropped image
        category_name = detection['categories'][0]['category_name']
        basepath = os.path.basename(image_path)
        save_path = os.path.join(output_directory, f'{basepath}_{counter}_{category_name}.jpg')  
        counter += 1
        Image.fromarray(cropped_image).save(save_path)

ami_ami_20230722000018-00-35.jpg
DetectionResult(detections=[Detection(bounding_box=BoundingBox(origin_x=223, origin_y=1541, width=328, height=247), categories=[Category(index=0, score=0.39453125, display_name='', category_name='moth')]), Detection(bounding_box=BoundingBox(origin_x=1835, origin_y=0, width=231, height=75), categories=[Category(index=1, score=0.30078125, display_name='', category_name='nonmoth')]), Detection(bounding_box=BoundingBox(origin_x=3387, origin_y=1699, width=140, height=129), categories=[Category(index=1, score=0.26953125, display_name='', category_name='nonmoth')]), Detection(bounding_box=BoundingBox(origin_x=965, origin_y=1888, width=172, height=201), categories=[Category(index=0, score=0.234375, display_name='', category_name='moth')]), Detection(bounding_box=BoundingBox(origin_x=3165, origin_y=331, width=251, height=157), categories=[Category(index=1, score=0.2109375, display_name='', category_name='nonmoth')]), Detection(bounding_box=BoundingBox(origin_x=1